<a href="https://www.kaggle.com/code/ibrahimnibrahim/tealeafbd-86-val-accuracy?scriptVersionId=244285723" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/tealeafbd-tea-leaf-disease-detection
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/3. Gray Blight
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/2. Brown Blight
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/7. Healthy leaf
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/6. Green mirid bug
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/1. Tea algal leaf spot
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/4. Helopeltis
/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD/5. Red spider


In [2]:
import tensorflow as tf

batch_size = 64
img_height = 224
img_width = 224
PATH='/kaggle/input/tealeafbd-tea-leaf-disease-detection/teaLeafBD/teaLeafBD'

train_ds,val_ds = tf.keras.utils.image_dataset_from_directory(
    PATH,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)


Found 5276 files belonging to 7 classes.
Using 4221 files for training.
Using 1055 files for validation.


In [3]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [4]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = strategy.experimental_distribute_dataset(train_ds)
val_ds = strategy.experimental_distribute_dataset(val_ds)

In [5]:
from tensorflow.keras.applications import EfficientNetV2B0

with strategy.scope():
    base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    base_model.trainable = False

    
    inputs = tf.keras.Input(shape=(img_height, img_width, 3))
    x = base_model(inputs, training=False)  # Pass inputs through the base model
    x = tf.keras.layers.Flatten()(x)  
    x = tf.keras.layers.Dense(256, activation='relu')(x)  # Fully connected layer
    outputs = tf.keras.layers.Dense(7, activation='softmax')(x)  # Output layer
    
    # Create the final model
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])

model.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetv2-b0 (Functional)       │ (None, 7, 7, 1280)          │       5,919,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 62720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      16,056,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,977,687 (83.84 MB)

 Trainable params: 16,058,375 (61.26 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs without improvement
    min_delta=0.001,     # Minimum change to qualify as improvement
    restore_best_weights=True  # Restore the best model weights
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation loss
    factor=0.5,         # Reduce learning rate by a factor of 0.1
    patience=3,         # Wait 3 epochs before reducing the learning rate
    min_lr=1e-6,        # Minimum learning rate
    mode='max',         # Minimize the monitored metric
    verbose=1           # Print a message when the learning rate is reduced
)

In [7]:
model.fit(train_ds, epochs=15,validation_data=val_ds,callbacks=[early_stopping,reduce_lr])

Epoch 1/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 35s 272ms/step - accuracy: 0.4478 - loss: 8.9639 - val_accuracy: 0.7689 - val_loss: 0.6800 - learning_rate: 0.0010
Epoch 2/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 8s 111ms/step - accuracy: 0.8373 - loss: 0.4694 - val_accuracy: 0.8049 - val_loss: 0.5756 - learning_rate: 0.0010
Epoch 3/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 8s 113ms/step - accuracy: 0.9006 - loss: 0.2949 - val_accuracy: 0.8106 - val_loss: 0.5519 - learning_rate: 0.0010
Epoch 4/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 109ms/step - accuracy: 0.9414 - loss: 0.1808 - val_accuracy: 0.7973 - val_loss: 0.6289 - learning_rate: 0.0010
Epoch 5/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 8s 113ms/step - accuracy: 0.9693 - loss: 0.1176 - val_accuracy: 0.8447 - val_loss: 0.5133 - learning_rate: 0.0010
Epoch 6/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 107ms/step - accuracy: 0.9772 - loss: 0.0881 - val_accuracy: 0.8333 - val_loss: 0.5149 - learning_rate: 0.0010
Epoch 7/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.9889 - loss: 0.0664 -